In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from rdkit import Chem
from rdkit.Chem.rdchem import HybridizationType, ChiralType
import pickle
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, matthews_corrcoef
from sklearn.model_selection import GridSearchCV

In [2]:
def find_topk(a, k, axis=-1, largest=True, sorted=True):
    if axis is None:
        axis_size = a.size
    else:
        axis_size = a.shape[axis]
    assert 1 <= k <= axis_size

    a = np.asanyarray(a)
    if largest:
        index_array = np.argpartition(a, axis_size-k, axis=axis)
        topk_indices = np.take(index_array, -np.arange(k)-1, axis=axis)
    else:
        index_array = np.argpartition(a, k-1, axis=axis)
        topk_indices = np.take(index_array, np.arange(k), axis=axis)
    topk_values = np.take_along_axis(a, topk_indices, axis=axis)
    if sorted:
        sorted_indices_in_topk = np.argsort(topk_values, axis=axis)
        if largest:
            sorted_indices_in_topk = np.flip(sorted_indices_in_topk, axis=axis)
        sorted_topk_values = np.take_along_axis(
            topk_values, sorted_indices_in_topk, axis=axis)
        sorted_topk_indices = np.take_along_axis(
            topk_indices, sorted_indices_in_topk, axis=axis)
        return sorted_topk_values, sorted_topk_indices
    return topk_values, topk_indices

# dataset

In [7]:
def _get_node_features(mol):
    all_node_feats = []

    identity = {
        'C':[1,0,0,0,0,0,0,0,0,0],
        'N':[0,1,0,0,0,0,0,0,0,0],
        'O':[0,0,1,0,0,0,0,0,0,0],
        'F':[0,0,0,1,0,0,0,0,0,0],
        'P':[0,0,0,0,1,0,0,0,0,0],
        'S':[0,0,0,0,0,1,0,0,0,0],
        'Cl':[0,0,0,0,0,0,1,0,0,0],
        'Br':[0,0,0,0,0,0,0,1,0,0],
        'I':[0,0,0,0,0,0,0,0,1,0],
        'other':[0,0,0,0,0,0,0,0,0,1],
    }
    for atom in mol.GetAtoms():
        node_feats = []
        # atom number
        idx = atom.GetIdx()
        # atom type one-hot 10
        node_feats.extend(identity.get(atom.GetSymbol(),[0,0,0,0,0,0,0,0,0,1]))
        # implicit valence
        node_feats.append(atom.GetImplicitValence())
        # formal charge
        node_feats.append(atom.GetFormalCharge())
        # radical electrons
        node_feats.append(atom.GetNumRadicalElectrons())
            
        # aromatic 0 or 1
        if atom.GetIsAromatic():
            node_feats.append(1)
        else:
            node_feats.append(0)

        # chirality
        chirality = atom.GetChiralTag()
        if chirality == ChiralType.CHI_TETRAHEDRAL_CCW: temp = [1, 0, 0, 0]
        if chirality == ChiralType.CHI_TETRAHEDRAL_CW: temp = [0, 1, 0, 0]
        if chirality == ChiralType.CHI_OTHER: temp = [0, 0, 1, 0]
        if chirality == ChiralType.CHI_UNSPECIFIED: temp = [0, 0, 0, 1]
        node_feats.extend(temp)
        # hybridization
        hybridization = atom.GetHybridization()
        if hybridization == HybridizationType.S: tmp = [1, 0, 0, 0, 0, 0, 0, 0]
        if hybridization == HybridizationType.SP: tmp = [0, 1, 0, 0, 0, 0, 0, 0]
        if hybridization == HybridizationType.SP2: tmp = [0, 0, 1, 0, 0, 0, 0, 0]
        if hybridization == HybridizationType.SP3: tmp = [0, 0, 0, 1, 0, 0, 0, 0]
        if hybridization == HybridizationType.SP3D: tmp = [0, 0, 0, 0, 1, 0, 0, 0]
        if hybridization == HybridizationType.SP3D2: tmp = [0, 0, 0, 0, 0, 1, 0, 0]
        if hybridization == HybridizationType.OTHER: tmp = [0, 0, 0, 0, 0, 0, 1, 0]
        if hybridization == HybridizationType.UNSPECIFIED: tmp = [0, 0, 0, 0, 0, 0, 0, 1]
        node_feats.extend(tmp)

        # degree
        node_feats.append(atom.GetDegree())
        # hydrogen
        node_feats.append(atom.GetTotalNumHs())
        # ring status 0 or 1
        if atom.IsInRing():
            node_feats.append(1)
        else:
            node_feats.append(0)
        # Append node features to matrix
        all_node_feats.append(node_feats)

    all_node_feats = np.asarray(all_node_feats, dtype=np.float)
    return all_node_feats

def _get_labels(mol):
    _y = []
    som = ['PRIMARY_SOM_1A2', 'PRIMARY_SOM_2A6','PRIMARY_SOM_2B6','PRIMARY_SOM_2C8','PRIMARY_SOM_2C9','PRIMARY_SOM_2C19','PRIMARY_SOM_2D6','PRIMARY_SOM_2E1','PRIMARY_SOM_3A4',
            'SECONDARY_SOM_1A2', 'SECONDARY_SOM_2A6','SECONDARY_SOM_2B6','SECONDARY_SOM_2C8','SECONDARY_SOM_2C9','SECONDARY_SOM_2C19','SECONDARY_SOM_2D6','SECONDARY_SOM_2E1','SECONDARY_SOM_3A4',
            'TERTIARY_SOM_1A2', 'TERTIARY_SOM_2A6','TERTIARY_SOM_2B6','TERTIARY_SOM_2C8','TERTIARY_SOM_2C9','TERTIARY_SOM_2C19','TERTIARY_SOM_2D6','TERTIARY_SOM_2E1','TERTIARY_SOM_3A4'
            ]
    result = []
    for k in som:
        try:
            _res = mol.GetProp(k)
            if ' ' in _res:
                res = _res.split(' ')
                for s in res:
                    result.append(int(s))
            else:
                result.append(int(_res))
        except:
            pass

    for data in result:
        _y.append(data)
    _y = list(set(_y))

    y = np.zeros(len(mol.GetAtoms()))
    for i in _y:
        y[i-1] = 1
    return y

In [8]:
filepath = '../Dataset/merged.sdf'
raws = Chem.SDMolSupplier(filepath)

In [9]:
mols = [mol for mol in raws]

In [10]:
dataset = []
for mol in mols:
    mol_feature = _get_node_features(mol)
    label = _get_labels(mol)
    dataset.append((mol_feature, label))

/tmp/ipykernel_1915300/2377401473.py:66: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  all_node_feats = np.asarray(all_node_feats, dtype=np.float)


In [11]:
import random
random.seed(42)
random.shuffle(dataset)

In [12]:
training_set = dataset[:int(len(dataset) * 0.8)]
test_set = dataset[int(len(dataset) * 0.8):]

In [13]:
tr_all = 0 # all training atoms
tr_indices = [] # molecule nums
y_train = [] # label
for feature, label in training_set:
    tr_all += feature.shape[0]
    tr_indices.append(len(label))
    y_train.extend(label)
print(f'tr_all means all training atoms num, num is {tr_all}')
print(f'training set molecule num is {len(tr_indices)}')
print(f'all atoms is {len(y_train)}')

tr_all means all training atoms num, num is 12010
training set molecule num is 544
all atoms is 12010


In [14]:
x_train = np.zeros((tr_all, training_set[0][0].shape[1]))
pre = 0
for i in range(len(tr_indices)):
    if i == 0:
        x_train[:tr_indices[i], :] = training_set[i][0]
    else:
        j = i - 1
        pre += tr_indices[j]
        x_train[pre:pre + tr_indices[i], :] = training_set[i][0]

In [15]:
y_train = np.array(y_train)

In [16]:
# test 时拼成一个大矩阵
te_all = 0
te_indices = []
y_test = []
for feature, label in test_set:
    te_all += feature.shape[0]
    te_indices.append(len(label))
    y_test.extend(label)
print(te_all)
print(len(te_indices))
print(len(y_test))

3259
136
3259


In [17]:
x_test = np.zeros((te_all, training_set[0][0].shape[1]))
pre = 0
for i in range(len(te_indices)):
    if i == 0:
        x_test[:te_indices[i], :] = test_set[i][0]
    else:
        j = i - 1
        pre += te_indices[j]
        x_test[pre:pre + te_indices[i], :] = test_set[i][0]

In [18]:
y_test = np.array(y_test)

# random forest

In [19]:
# gridsearch random forest
max_depths = [10, 20, 30]
criterions = ['gini', 'entropy']
_n_estimators = [100, 150, 200, 50]
tuned_parameters = dict(max_depth=max_depths,criterion=criterions, n_estimators=_n_estimators)
clf = RandomForestClassifier(random_state=42, class_weight='balanced')
grid = GridSearchCV(clf, tuned_parameters, cv=5, scoring='roc_auc')
grid.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [10, 20, 30],
                         'n_estimators': [100, 150, 200, 50]},
             scoring='roc_auc')

In [20]:
print(grid.best_score_)
print(grid.best_params_)

0.7943690667280146
{'criterion': 'entropy', 'max_depth': 20, 'n_estimators': 200}


In [21]:
clf = RandomForestClassifier(n_estimators=200,max_depth=20,criterion='entropy',random_state=42, class_weight='balanced').fit(x_train, y_train)

In [22]:
preds = clf.predict(x_test)
logits = clf.predict_proba(x_test)

In [23]:
# 拆分出logits， 计算top-k
top2 = 0
pre = 0
for i in range(len(te_indices)):
    if i == 0:
        # 取出第一列，
        y_preds = logits[:te_indices[i], :][:, 1]
        print(y_preds.shape)
        topk_values, topk_indices = find_topk(y_preds, 2)
        for a in range(len(test_set[i][1])):
            if test_set[i][1][a] == 1 and a in topk_indices:
                top2 += 1
                break
    else:
        j = i - 1
        pre += te_indices[j]
        y_preds = logits[pre:pre + te_indices[i], :][:, 1]
        topk_values, topk_indices = find_topk(y_preds, 2)
        for a in range(len(test_set[i][1])):
            if test_set[i][1][a] == 1 and a in topk_indices:
                top2 += 1
                break
print(top2)
print(len(test_set))
print(f'top2 acc: {top2 / len(test_set)}')

(21,)
70
136
top2 acc: 0.5147058823529411


In [24]:
roc_auc_score(y_test, preds)

0.7353808756891919

In [25]:
matthews_corrcoef(y_test, preds)

0.2790040765620911

# decision tree

In [26]:
# gridsearch
max_depths = [10, 20, 30, 40, 50, None]
criterions = ['gini', 'entropy']
splitters = ['best', 'random']
tuned_parameters = dict(max_depth=max_depths,criterion=criterions, splitter=splitters)
clf = DecisionTreeClassifier(random_state=42, class_weight='balanced')
grid = GridSearchCV(clf, tuned_parameters, cv=5, scoring='roc_auc')
grid.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [10, 20, 30, 40, 50, None],
                         'splitter': ['best', 'random']},
             scoring='roc_auc')

In [27]:
print(grid.best_score_)
print(grid.best_params_)

0.7934739454526759
{'criterion': 'gini', 'max_depth': 20, 'splitter': 'best'}


In [28]:
clf = DecisionTreeClassifier(random_state=42, class_weight='balanced', criterion='gini', max_depth=20, splitter='best').fit(x_train, y_train)

In [29]:
preds = clf.predict(x_test)
logits = clf.predict_proba(x_test)

In [30]:
roc_auc_score(y_test, preds)

0.7371013087944208

In [31]:
matthews_corrcoef(y_test, preds)

0.28105320254767735

In [32]:
# 拆分出logits， 计算top-k
top2 = 0
pre = 0
for i in range(len(te_indices)):
    if i == 0:
        y_preds = logits[:te_indices[i], :][:, 1]
        topk_values, topk_indices = find_topk(y_preds, 2)
        for a in range(len(test_set[i][1])):
            if test_set[i][1][a] == 1 and a in topk_indices:
                top2 += 1
                break
    else:
        j = i - 1
        pre += te_indices[j]
        y_preds = logits[pre:pre + te_indices[i], :][:, 1]
        topk_values, topk_indices = find_topk(y_preds, 2)
        for a in range(len(test_set[i][1])):
            if test_set[i][1][a] == 1 and a in topk_indices:
                top2 += 1
                break
print(top2)
print(len(test_set))
print(f'top2 acc: {top2 / len(test_set)}')

71
136
top2 acc: 0.5220588235294118


# logistic regression

In [33]:
# gridsearch
Cs = [0.0001, 0.0005, 0.001, 0.005, 0.1, 0.5, 1, 10, 50, 100, 150, 200]
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
tuned_parameters = dict(C=Cs, solver=solvers)
clf = LogisticRegression(random_state=42, class_weight='balanced', max_iter=100000, n_jobs=-1)
grid = GridSearchCV(clf, tuned_parameters, cv=5, scoring='roc_auc')
grid.fit(x_train, y_train)

/home/jgl/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1355: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 32.
  warnings.warn("'n_jobs' > 1 does not have any effect when"
/home/jgl/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1355: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 32.
  warnings.warn("'n_jobs' > 1 does not have any effect when"
/home/jgl/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1355: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 32.
  warnings.warn("'n_jobs' > 1 does not have any effect when"
/home/jgl/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1355: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 

/home/jgl/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1355: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 32.
  warnings.warn("'n_jobs' > 1 does not have any effect when"
/home/jgl/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1355: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 32.
  warnings.warn("'n_jobs' > 1 does not have any effect when"
/home/jgl/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1355: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 32.
  warnings.warn("'n_jobs' > 1 does not have any effect when"
/home/jgl/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1355: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 

GridSearchCV(cv=5,
             estimator=LogisticRegression(class_weight='balanced',
                                          max_iter=100000, n_jobs=-1,
                                          random_state=42),
             param_grid={'C': [0.0001, 0.0005, 0.001, 0.005, 0.1, 0.5, 1, 10,
                               50, 100, 150, 200],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='roc_auc')

In [34]:
print(grid.best_score_)
print(grid.best_params_)

0.7795044067643004
{'C': 10, 'solver': 'sag'}


In [35]:
clf = LogisticRegression(random_state=42,C=10,solver="sag", class_weight='balanced', max_iter=100000, n_jobs=-1).fit(x_train, y_train)

In [36]:
preds = clf.predict(x_test)
logits = clf.predict_proba(x_test)

In [37]:
roc_auc_score(y_test, preds)

0.6954023851239634

In [38]:
matthews_corrcoef(y_test, preds)

0.26324524329177096

In [39]:
# 拆分出logits， 计算top-k
top2 = 0
pre = 0
for i in range(len(te_indices)):
    if i == 0:
        y_preds = logits[:te_indices[i], :][:, 1]
        topk_values, topk_indices = find_topk(y_preds, 2)
        for a in range(len(test_set[i][1])):
            if test_set[i][1][a] == 1 and a in topk_indices:
                top2 += 1
                break
    else:
        j = i - 1
        pre += te_indices[j]
        y_preds = logits[pre:pre + te_indices[i], :][:, 1]
        topk_values, topk_indices = find_topk(y_preds, 2)
        for a in range(len(test_set[i][1])):
            if test_set[i][1][a] == 1 and a in topk_indices:
                top2 += 1
                break
print(top2)
print(len(test_set))
print(f'top2 acc: {top2 / len(test_set)}')

75
136
top2 acc: 0.5514705882352942


# naive bayes

In [40]:
clf = GaussianNB()
clf.fit(x_train, y_train)

GaussianNB()

In [41]:
preds = clf.predict(x_test)
logits = clf.predict_proba(x_test)

In [42]:
roc_auc_score(y_test, preds)

0.5706315796245821

In [43]:
matthews_corrcoef(y_test, preds)

0.11947133943677866

In [44]:
# 拆分出logits， 计算top-k
top2 = 0
pre = 0
for i in range(len(te_indices)):
    if i == 0:
        y_preds = logits[:te_indices[i], :][:, 1]
        topk_values, topk_indices = find_topk(y_preds, 2)
        for a in range(len(test_set[i][1])):
            if test_set[i][1][a] == 1 and a in topk_indices:
                top2 += 1
                break
    else:
        j = i - 1
        pre += te_indices[j]
        y_preds = logits[pre:pre + te_indices[i], :][:, 1]
        topk_values, topk_indices = find_topk(y_preds, 2)
        for a in range(len(test_set[i][1])):
            if test_set[i][1][a] == 1 and a in topk_indices:
                top2 += 1
                break
print(top2)
print(len(test_set))
print(f'top2 acc: {top2 / len(test_set)}')

68
136
top2 acc: 0.5


# svm

In [45]:
# gridsearch
Cs = [0.0001, 0.0005, 0.001, 0.005, 0.1, 0.5, 1, 10, 50, 100, 150, 200]
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
degrees = [2,3,4,5]
gammas = ['scale', 'auto']
tuned_parameters = dict(C=Cs,kernel=kernels, degree=degrees, gamma = gammas)
clf = SVC(random_state=42, class_weight='balanced')
grid = GridSearchCV(clf, tuned_parameters, cv=5, scoring='roc_auc')
grid.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=SVC(class_weight='balanced', random_state=42),
             param_grid={'C': [0.0001, 0.0005, 0.001, 0.005, 0.1, 0.5, 1, 10,
                               50, 100, 150, 200],
                         'degree': [2, 3, 4, 5], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='roc_auc')

In [46]:
print(grid.best_score_)
print(grid.best_params_)

0.7704019157268416
{'C': 0.1, 'degree': 2, 'gamma': 'scale', 'kernel': 'linear'}


In [47]:
clf = SVC(C=0.1, degree=2, gamma='scale', kernel='linear',random_state=42, class_weight='balanced', probability=True).fit(x_train, y_train)

In [48]:
preds = clf.predict(x_test)
logits = clf.predict_proba(x_test)

In [49]:
roc_auc_score(y_test, preds)

0.6858373346314767

In [50]:
matthews_corrcoef(y_test, preds)

0.2462519243916313

In [51]:
# 拆分出logits， 计算top-k
top2 = 0
pre = 0
for i in range(len(te_indices)):
    if i == 0:
        y_preds = logits[:te_indices[i], :][:, 1]
        topk_values, topk_indices = find_topk(y_preds, 2)
        for a in range(len(test_set[i][1])):
            if test_set[i][1][a] == 1 and a in topk_indices:
                top2 += 1
                break
    else:
        j = i - 1
        pre += te_indices[j]
        y_preds = logits[pre:pre + te_indices[i], :][:, 1]
        topk_values, topk_indices = find_topk(y_preds, 2)
        for a in range(len(test_set[i][1])):
            if test_set[i][1][a] == 1 and a in topk_indices:
                top2 += 1
                break
print(top2)
print(len(test_set))
print(f'top2 acc: {top2 / len(test_set)}')

69
136
top2 acc: 0.5073529411764706
